# Detección de Video

Para ejecutar el siguiente código, es necesario descargarse 2 archivos: El dataset 'sign_dataset.csv' y el modelo de Red Neuronal entrenado 'sign_model.h5'

También es necesario ejecutarlo en local y tener conectado una webcam pues va a conectarse a esta.

# Importar Dependecias

In [1]:
# Importa dependencias
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn

In [2]:
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd

from tensorflow.keras.models import load_model

In [3]:
# Modelos de mediapipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Funciones para el uso de Mediapipe

In [4]:
# Funcion que devuelve una array de keypoints a partir de un result de mediapipe
def extract_keypoints(results):
    return np.array([[res.x, res.y, res.z] for res in results.multi_hand_landmarks[0].landmark]).flatten()

In [5]:
# Funcion que procesa la imagen y detecta los keypoints de la mano
def mediapipe_detection(image, hands):
    keypoints = None
    left_hand = None
    
    image = cv2.flip(image, 1)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    if results.multi_hand_landmarks is not None:
        if results.multi_handedness[0].classification[0].label == 'Left':
            left_hand = True
            image_rgb = cv2.flip(image_rgb, 1)
            results = hands.process(image_rgb)
        if results.multi_hand_landmarks is not None:
            keypoints = extract_keypoints(results)
            
            if left_hand is True: image = cv2.flip(image, 1)
                
            # Dibuja los keypoints
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
            if left_hand is True: image = cv2.flip(image, 1)
    return image, results, keypoints

# Carga del Modelo Preentrenado

In [6]:
# Carga el modelo preentrenado y el dataset con los labels
modelo = load_model('sign_model.h5')
df = pd.read_csv('sign_dataset.csv', index_col=0)
label_names = df['label'].unique()
label_names_sorted = np.sort(label_names)

# Reconocimiento de Signos

In [23]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands=1) as hands:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        if ret == False: break

        # Make detections
        image, results, keypoints = mediapipe_detection(frame, hands)
        
        # Hace la prediccion a partir de los keypoints
        letra = ''
        acc = ''
        if keypoints is not None:
            Ypr = modelo.predict(keypoints.reshape(1, 63))
            letra = label_names_sorted[np.argmax(Ypr)]
            acc = f'Acc: {str(np.max(Ypr))}'
        # Dibuja la prediccion en la imagen
        image = cv2.flip(image, 1)
        cv2.rectangle(image, (0, 0), (120, 120), (245, 117, 16), -1)
        cv2.putText(image, letra, (30, 95), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, acc, (2, 135), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)
        
        cv2.putText(image, 'Pulsa "q" para cerrar la ventana', (420, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.4,  (155, 155, 155), 1)
        
        # Muestra la imagen
        cv2.imshow('OpenCV Feed', image)

        # Cierra el bucle
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()